## The Decision Problem

We make widgets. Have a set of production facilities that produce boxes of widgets. There is also a set of distribution locations that will then distribute the widgets for sale. Each distribution center has a forecasted demand and each production facility has a min and max number of widgets it can make during this period. We need to ensure that each distribution facility receives enough widgets to satisfy demand from production and we want to do this at minimal cost. The minimum production is 75% of the production facilities max value.

## Sets

Our sets are:
*   $P = \{\textrm{`Baltimore'}, \textrm{`Cleveland'}, \textrm{`Little Rock'}, \textrm{`Birmingham'}, \textrm{`Charleston'} \}$
*   $D = \{\textrm{`Columbia'}, \textrm{`Indianapolis'}, \textrm{`Lexington'}, \textrm{`Nashville'}, \textrm{`Richmond'}, \textrm{`St. Louis'} \}$



In [1]:
#!pip install gurobipy

In [2]:
# Import packages
import pandas as pd
import gurobipy as gb
from gurobipy import GRB

In [3]:
# Define sets P and D respectively
production = ['Baltimore', 'Cleveland', 'Little_Rock', 'Birmingham', 'Charleston']
distribution = ['Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St_Louis']

In [4]:
# Define the gurobipy model for the decision problem
m = gb.Model('widgets')

Restricted license - for non-production use only - expires 2026-11-23


## Parameters

*   $m_{p}$ is the max production in location $p, \forall p \in P$ $\hspace{6.7cm} \texttt{max\_prod[p]}$
*   $n_{d}$ is the number of customers for a distribution center $d, \forall d \in D$ $\hspace{3.7cm} \texttt{n\_demand[d]}$
*   $c_{p,d}$ is the cost to ship a widget between location $p$ and location $d$, $\forall p \in P, d \in D$ $\hspace{1cm} \texttt{cost[p, d]}$

In [5]:
# Use squeeze=True to make the costs a series
# Downloaded from https://github.com/Gurobi/modeling-examples/tree/master
# path = "drive/MyDrive/Colab Notebooks/gurobi_101/"
path = ""
transp_cost = pd.read_csv(path + "cost.csv", index_col=[0, 1]).squeeze()

In [6]:
# Pivot to view the costs a bit easier
transp_cost.reset_index().pivot(index='production', columns='distribution', values='cost')

distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St_Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little_Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [7]:
max_prod = pd.Series([180, 200, 140, 80, 180], index=production, name="max_production")
n_demand = pd.Series([89, 95, 121, 101, 116, 181], index=distribution, name="demand")

In [8]:
max_prod.to_frame()

,max_production
Baltimore,180
Cleveland,200
Little_Rock,140
Birmingham,80
Charleston,180


In [9]:
n_demand.to_frame()

,demand
Columbia,89
Indianapolis,95
Lexington,121
Nashville,101
Richmond,116
St_Louis,181


We also have the requirement that each production facility needs to produce at 75% of this maximum output. We'll denote this value by $a$ in the formulation "frac" for the fraction of maximum production required. Initially we set $a = 0.75$.

In [10]:
frac = 0.75

## Decision Variables
The decision here is to determine the number of boxes to send from each production facility to each distribution location. Decision variables come in three main flavors:

- `Continuous`: Price of a product
- `Integer`: The number of food trucks to use for an event
- `Binary`: Yes/No decision to include a certain stock in a portfolio

Decision variables (and parameters) are indexed using elements of sets that we define for the problem. In this example, let's start with a set of cities that produce our widget, which we call set $P$ for the formulation but can define as 'production' in the code. And a set of cities that distribute the widget $D$ and 'distribution' similarly. The decision here is to determine the number of boxes to send from each production facility to each distribution location.

Let $x_{p, d}$ be the number of widgets that are produced at facility $p$ and shipped to location $d$.

## Add Variables in gurobipy
$\texttt{gurobipy}$ let's you add decision variables primarily with two (similar) commands:

*   $\texttt{addVar()}$ adds a single variable
*   $\texttt{addVars()}$ adds a group of variables by sets/indices

In [11]:
# Loop through each p and d combination to create a decision variable
x = {}
for p in production:
    for d in distribution:
        x[p, d] = m.addVar(name = p+"_to_"+d)
m.update() # Updates the model to include any changes that have been made

In [12]:
x

{('Baltimore', 'Columbia'): <gurobi.Var Baltimore_to_Columbia>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var Baltimore_to_Indianapolis>,
 ('Baltimore', 'Lexington'): <gurobi.Var Baltimore_to_Lexington>,
 ('Baltimore', 'Nashville'): <gurobi.Var Baltimore_to_Nashville>,
 ('Baltimore', 'Richmond'): <gurobi.Var Baltimore_to_Richmond>,
 ('Baltimore', 'St_Louis'): <gurobi.Var Baltimore_to_St_Louis>,
 ('Cleveland', 'Columbia'): <gurobi.Var Cleveland_to_Columbia>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var Cleveland_to_Indianapolis>,
 ('Cleveland', 'Lexington'): <gurobi.Var Cleveland_to_Lexington>,
 ('Cleveland', 'Nashville'): <gurobi.Var Cleveland_to_Nashville>,
 ('Cleveland', 'Richmond'): <gurobi.Var Cleveland_to_Richmond>,
 ('Cleveland', 'St_Louis'): <gurobi.Var Cleveland_to_St_Louis>,
 ('Little_Rock', 'Columbia'): <gurobi.Var Little_Rock_to_Columbia>,
 ('Little_Rock', 'Indianapolis'): <gurobi.Var Little_Rock_to_Indianapolis>,
 ('Little_Rock', 'Lexington'): <gurobi.Var Little_Rock_to_Lexing

In [13]:
# Alternate way to specify variables
# x = m.addVars(production, distribution, name="prod_ship")
# m.update()

In [14]:
# Yet another way to specify variables
# x = m.addVars(transp_cost.index, name="prod_ship")
# m.update()

## Constraints
Adding constraints to a model is similar to adding variables

*   $\texttt{addConstr()}$ adds a single constraint
*   $\texttt{addConstrs()}$ adds a group of constraints using a Python generator expression

We'll formulate the demand constraints for each distribution location first and add them to the model

$$
\sum_{p} x_{p,d} \ge n_{d}, \hspace{0.5cm} \forall\, d \in D \hspace{1cm} \texttt{meet\_demand[d]}
$$

This will be the first time we use $\texttt{gb.quicksum()}$. There are other ways to sum expressions in gurobipy and while this method isn't the most concise to code, it is easy to compare it to the summation in the formulation to see how it works.


In [15]:
# Adding all the demand constraints
meet_demand = m.addConstrs((gb.quicksum(x[p, d] for p in production) >= n_demand[d] for d in distribution),
                           name = "meet_demand")
m.update()

In [16]:
meet_demand

{'Columbia': <gurobi.Constr meet_demand[Columbia]>,
 'Indianapolis': <gurobi.Constr meet_demand[Indianapolis]>,
 'Lexington': <gurobi.Constr meet_demand[Lexington]>,
 'Nashville': <gurobi.Constr meet_demand[Nashville]>,
 'Richmond': <gurobi.Constr meet_demand[Richmond]>,
 'St_Louis': <gurobi.Constr meet_demand[St_Louis]>}

Next we have the maximum number of widgets each production facility can make. We also have that each facility must make at least 75% of its max production.

$$
\begin{align}
\sum_{d} x_{p, d} \leq m_{p}, \hspace{1.9cm} &\forall\, p \in P \hspace{1cm} \texttt{can\_produce[p]}\\
\sum_{d} x_{p, d} \geq a \times m_{p}, \hspace{1cm} &\forall\, p \in P \hspace{1cm} \texttt{must\_produce[p]}
\end{align}
$$

In [17]:
# Adding all production constraints
can_produce = m.addConstrs((gb.quicksum(x[p, d] for d in distribution) <= max_prod[p] for p in production),
                           name = "can_produce")
must_produce = m.addConstrs((gb.quicksum(x[p, d] for d in distribution) >= frac * max_prod[p] for p in production),
                             name = "must_produce")
m.update()

In [18]:
can_produce

{'Baltimore': <gurobi.Constr can_produce[Baltimore]>,
 'Cleveland': <gurobi.Constr can_produce[Cleveland]>,
 'Little_Rock': <gurobi.Constr can_produce[Little_Rock]>,
 'Birmingham': <gurobi.Constr can_produce[Birmingham]>,
 'Charleston': <gurobi.Constr can_produce[Charleston]>}

## Objective Function
This is done using `setObjective()`. The second argument (in this case
`GRB.MINIMIZE`) is called the model's *sense*. For maximization problem we would use `GRB.MAXIMIZE`.

$$
\textrm{minimize} \sum_{p, d} c_{p, d} x_{p, d} \hspace{1cm} \forall\, p \in P, d \in D
$$



In [19]:
# Set objective to minimize the cost of transportation
m.setObjective(gb.quicksum(transp_cost[p, d] * x[p, d] for p in production for d in distribution),
               GRB.MINIMIZE)
m.update()

## Find, Extract, and Analyze the Solution
Before running the optimization, it is a good idea to write an lp file. This is a text file that prints out the variables, constraints, and objective like we would see in the *formulation*, just without the summation symbols and using the names we designated

In [20]:
m.write(path + 'widget_shipment.lp')

In [21]:
# Run the Optimization
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x20186c14
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.704890000e+03


## Extract the Solution
There are many ways to get the values of decision variables out of gurobipy

In [22]:
# The uppercase 'X' is the attribute of the decision variable 'x'
x_values = pd.Series(m.getAttr('X', x), name="shipment", index=transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
sol

cost  shipment
production  distribution                
Baltimore   Columbia      4.50       0.0
            Indianapolis  5.09       0.0
            Lexington     4.33       0.0
            Nashville     5.96      19.0
            Richmond      1.96     116.0
            St_Louis      7.30       0.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Lexington     2.54       0.0
            Nashville     4.13       2.0
            Richmond      3.20       0.0
            St_Louis      4.88       0.0
Little_Rock Columbia      6.42       0.0
            Indianapolis  4.83       0.0
            Lexington     3.39       0.0
            Nashville     4.40       0.0
            Richmond      7.44       0.0
            St_Louis      2.92     140.0
Birmingham  Columbia      3.33       0.0
            Indianapolis  4.33       0.0
            Lexington     3.38       0.0
            Nashville     1.53      80.0
            Richmond      5.95       0.0
            St_Louis      4.01       0.0
Charleston  Columbia      3.02       0.0
            Indianapolis  2.61       0.0
            Lexington     1.61     121.0
            Nashville     4.44       0.0
            Richmond      2.36       0.0
            St_Louis      4.60      41.0

In [23]:
sol[sol.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      19.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Nashville     4.13       2.0
Little_Rock St_Louis      2.92     140.0
Birmingham  Nashville     1.53      80.0
Charleston  Lexington     1.61     121.0
            St_Louis      4.60      41.0

In [24]:
# You can the name and value of all the decision variables:
all_vars = {v.varName: v.x for v in m.getVars()}
all_vars

{'Baltimore_to_Columbia': 0.0,
 'Baltimore_to_Indianapolis': 0.0,
 'Baltimore_to_Lexington': 0.0,
 'Baltimore_to_Nashville': 19.0,
 'Baltimore_to_Richmond': 116.0,
 'Baltimore_to_St_Louis': 0.0,
 'Cleveland_to_Columbia': 89.0,
 'Cleveland_to_Indianapolis': 95.0,
 'Cleveland_to_Lexington': 0.0,
 'Cleveland_to_Nashville': 2.0,
 'Cleveland_to_Richmond': 0.0,
 'Cleveland_to_St_Louis': 0.0,
 'Little_Rock_to_Columbia': 0.0,
 'Little_Rock_to_Indianapolis': 0.0,
 'Little_Rock_to_Lexington': 0.0,
 'Little_Rock_to_Nashville': 0.0,
 'Little_Rock_to_Richmond': 0.0,
 'Little_Rock_to_St_Louis': 140.0,
 'Birmingham_to_Columbia': 0.0,
 'Birmingham_to_Indianapolis': 0.0,
 'Birmingham_to_Lexington': 0.0,
 'Birmingham_to_Nashville': 80.0,
 'Birmingham_to_Richmond': 0.0,
 'Birmingham_to_St_Louis': 0.0,
 'Charleston_to_Columbia': 0.0,
 'Charleston_to_Indianapolis': 0.0,
 'Charleston_to_Lexington': 121.0,
 'Charleston_to_Nashville': 0.0,
 'Charleston_to_Richmond': 0.0,
 'Charleston_to_St_Louis': 41.0}

In [25]:
# Or we can interate over a specific variable and only return values that are of interest to us.
xvals = {k: v.x for k, v in x.items() if v.x > 0}
xvals

{('Baltimore', 'Nashville'): 19.0,
 ('Baltimore', 'Richmond'): 116.0,
 ('Cleveland', 'Columbia'): 89.0,
 ('Cleveland', 'Indianapolis'): 95.0,
 ('Cleveland', 'Nashville'): 2.0,
 ('Little_Rock', 'St_Louis'): 140.0,
 ('Birmingham', 'Nashville'): 80.0,
 ('Charleston', 'Lexington'): 121.0,
 ('Charleston', 'St_Louis'): 41.0}

## Solution Analysis

While determining the optimal transportation of widgets was our goal, we may want to dig a little deaper into the solution. For example, we can aggregate the total production by facility to see which locations (if any) did not produce their maximum capacity of widgets and which (if any) production facilities are at the lower bound of their production.

In [26]:
# Sum the shipment amount by production facility
ship_out = sol.groupby('production')['shipment'].sum()
ship_out_df = pd.DataFrame({
    'Remaining': max_prod - ship_out,
    'Utilization': ship_out/max_prod,
    'Max Capacity': max_prod
})
ship_out_df.head(10)

,Remaining,Utilization,Max Capacity
Baltimore,45.0,0.75,180
Birmingham,0.0,1.00,80
Charleston,18.0,0.90,180
Cleveland,14.0,0.93,200
Little_Rock,0.0,1.00,140


In mathematical optimization, when the left-hand and the right-hand sides of an inequality constraint are equal, we say the constraint is *binding*. When this *doesn't happen* then there is **slack** or **surplus** in that constraint. We can get this value by calling the `Slack` attribute of a constraint.

In [27]:
slack_df = pd.DataFrame({
    'Remaining': [can_produce[p].Slack for p in production],
    'Utilization': [1-can_produce[p].Slack/max_prod[p] for p in production]
}, index=production)

In [28]:
slack_df.head(10)

,Remaining,Utilization
Baltimore,45.0,0.75
Cleveland,14.0,0.93
Little_Rock,0.0,1.00
Birmingham,0.0,1.00
Charleston,18.0,0.90


## Using Binary Variables

Binary variables are used to choose alternatives in mathematical optimization. They can be interpreted as a yes/no or a an on/off switch.

In the original problem Birmingham's production is much lower than the rest of the facilities. Suppose we have the option to expand that facilities max capacity by either 25 or 50 widgets, but there is a cost of \\$50 and \\$70, respectively, to choose one of these options and we can choose at most one. We'll use a binary decision variable for each option named $xprod$.

Let $xprod_{0} = 1$ if we choose the first option and expand production capacity by 25 and 0 otherwise. Let $xprod_{1} = 1$ if we choose the second option and expand production capacity by 50 and 0 otherwise.

While it's fairly common to use single lowercase letters as decision variables, it is not necessary and you'll see variables defined as above (where they are more descriptive) quite often. We will formulate a new model that contains the same decision variables and demand constraints as before.

In [29]:
# We use m2 for the second model
m2 = gb.Model('widget2')

x = {}
for p in production:
    for d in distribution:
        x[p, d] = m2.addVar(name = p+"_to_"+d)

#x = m2.addVars(production, distribution, obj = transp_cost, name='prod_ship')
m2.setObjective(gb.quicksum(transp_cost[p, d] * x[p, d] for p in production for d in distribution),
                GRB.MINIMIZE)
    
meet_demand = m2.addConstrs((gb.quicksum(x[p, d] for p in production) >= n_demand[d] for d in distribution), name="meet_demand")
m2.update()

In the cell above we can also use `x = m2.addVars(production, distribution, obj = transp_cost, name='prod_ship')`. This will set the coefficient of the added decision variables in the objective function and is equivalent to what we did earlier by attaching the transportation costs between each production and distribution location to the appropriate decision variable.

Next, we'll add the same constraints for production limits as before for each production facility other than Birmingham. The formulation is basically the same other than the set the constraints hold for.
$$
\begin{align}
\sum_{d} x_{p, d} \leq m_{p}, \hspace{1.9cm} &\forall p \in P - {\textrm{Birmingham}} \\
\sum_{d} x_{p, d} \geq a \times m_{p}, \hspace{1cm} &\forall p \in P - {\textrm{Birmingham}} \\
\end{align}
$$

In gurobipy, this is done by adding a condition in the generator expression

In [30]:
can_produce = m2.addConstrs((gb.quicksum(x[p, d] for d in distribution) <= max_prod[p] for p in production if p != "Birmingham"),
                           name="can_produce")
must_produce = m2.addConstrs((gb.quicksum(x[p, d] for d in distribution) >= frac * max_prod[p] for p in production if p != "Birmingham"),
                             name="must_produce")

Now, we add the new binary variables

In [31]:
b_prod_costs = {0: 50, 1: 70}
xprod = m2.addVars(range(2), vtype=GRB.BINARY, obj=b_prod_costs, name='expand_birmingham_prod')

m2.update()

Let's breakdown each of the arguments in the cell above - there are a few new things here.

1. `range(n)` is used to add $n$ decision variables. In this case we add two variables.
2. We need to declare this as a binary variable using `vtype`.
3. We again use the `obj` capability to immediately set the objective function coefficients for these variables

The objective and new binary variables look like this in the formulation:
$$
\begin{align}
\textrm{minimize} &\sum_{p, d} c_{p, d}x_{p, d} + 50 \times xprod_{0} + 75 \times xprod_{1} \hspace{0.5cm} \forall \, p \in P,\, d \in D\\
& xprod_{i} \in \{0, 1\}, \hspace{5cm} \textrm{for}\,\, i \in \{0, 1\}
\end{align}
$$

Next we have the production constraints that are specific to the Birmingham facility

$$
\begin{align}
\sum_{d} &x_{p, d} \leq m_{p} + 25 \times xprod_{0} + 50 \times xprod_{1}, \hspace{2cm} p = \textrm{Birmingham}\\
\sum_{d} &x_{p, d} \geq a \times (m_{p} + 25 \times xprod_{0} + 50 \times xprod_{3}), \hspace{1cm} p = \textrm{Birmingham}\\
\end{align}
$$

In [32]:
birmingham_max = m2.addConstr(gb.quicksum(x['Birmingham', d] for d in distribution) <= max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1],
                              name='birmingham_add_max')
birmingham_min = m2.addConstr(gb.quicksum(x['Birmingham', d] for d in distribution) >= frac*(max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1]),
                              name='birmingham_add_min')
m2.update()

It was stated above that we can select at most one of the expansion options which means we cannot allow both $xprod_{0}$ and $xprod_{x}$ to equal to one. To model this we add a constraint limiting the sum of these two binary variables to at most one
$$
\sum_{x} xprod_{i} \leq 1
$$
The corresponding constraint in gurobipy:

In [33]:
birmingham_lim = m2.addConstr(gb.quicksum(xprod[i] for i in range(2)) <= 1, name = 'expansion_choice')
m2.update()

In [35]:
m2.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 17 rows, 32 columns and 96 nonzeros
Model fingerprint: 0xe9f5de1e
Variable types: 30 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve time: 0.06s
Presolved: 17 rows, 32 columns, 96 nonzeros
Variable types: 30 continuous, 2 integer (2 binary)
Found heuristic solution: objective 1704.8900000

Root relaxation: objective 1.694760e+03, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1694.76000    0    1 1704.89000

In [36]:
obj1 = m.getObjective()
obj2 = m2.getObjective()
print(f"The original model had a total cost of {round(obj1.getValue(), 2)}")
print(f"The new formulation has a total cost of {round(obj2.getValue(), 2)}")

The original model had a total cost of 1704.89
The new formulation has a total cost of 1700.4


What does the change in objective function value tell us?

Let's look at values of our binary variables

In [37]:
pd.Series(m2.getAttr('X', xprod))

0    1.0
1   -0.0
dtype: float64

The model selected the first expansion option since $xprod_{0} = 1$, which was increasing production by 25 widgets in Birmingham. We can see the rest of the solution, which will include the increase in Birmingham's production capacity.

In [42]:
x2_values = pd.Series(m2.getAttr('X', x), name="shipment", index=transp_cost.index)
sol2 = pd.concat([transp_cost, x2_values], axis=1)
sol2[sol2.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Richmond      1.96     135.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little_Rock St_Louis      2.92     140.0
Birmingham  Nashville     1.53     101.0
            St_Louis      4.01       4.0
Charleston  Lexington     1.61     121.0
            St_Louis      4.60      37.0

In [45]:
# Sum the shipment amount by production facility
ship_out2 = sol2.groupby('production')['shipment'].sum()
ship_out2.head()

production
Baltimore      135.0
Birmingham     105.0
Charleston     158.0
Cleveland      184.0
Little_Rock    140.0
Name: shipment, dtype: float64

In [54]:
# Update max_prod based on xprod
max_prod2 = max_prod.copy()
max_prod2['Birmingham'] = max_prod2['Birmingham'] + + 25*xprod[0].X + 50*xprod[1].X
max_prod2

Baltimore      180
Cleveland      200
Little_Rock    140
Birmingham     105
Charleston     180
Name: max_production, dtype: int64

In [56]:
ship_out_df2 = pd.DataFrame({
    'Remaining': max_prod2 - ship_out2,
    'Utilization': round(ship_out2/max_prod2, 2),
    'Max Capacity': max_prod2
})
ship_out_df2.head(10)

,Remaining,Utilization,Max Capacity
Baltimore,45.0,0.75,180
Birmingham,0.0,1.00,105
Charleston,22.0,0.88,180
Cleveland,16.0,0.92,200
Little_Rock,0.0,1.00,140


In [57]:
ship_out_df.head(10)

,Remaining,Utilization,Max Capacity
Baltimore,45.0,0.75,180
Birmingham,0.0,1.00,80
Charleston,18.0,0.90,180
Cleveland,14.0,0.93,200
Little_Rock,0.0,1.00,140
